In [1]:
import os
import pysam

In [2]:
def make_het_snp_bed(f_vcf, f_bed):
    rows = []
    with pysam.VariantFile(f_vcf) as f:
        sample = list(f.header.samples)[0]
        for record in f:
            gt = record.samples[sample]["GT"]
            a1, a2 = record.alleles[gt[0]], record.alleles[gt[1]]
            if len(a1) != 1 or len(a2) != 1 or a1 == a2:
                continue
            start = record.start
            end = record.stop
            if end - start != 1:
                continue
            rows.append([record.contig, start, end, "%s|%s" % (a1, a2)])
    rows.sort()
    with open(f_bed[:-3], "w+") as fw:
        for row in rows:
            fw.write("\t".join(map(str, row)) + "\n")
    cmd = "bgzip -f %s" % f_bed[:-3]
    assert os.system(cmd) == 0
    cmd= "tabix -p bed -f %s" % f_bed
    assert os.system(cmd) == 0

In [9]:
f_vcf = "/lustre/grp/tfclab/chenzg/repositories/GRCh38_HG001_SNP_Indel/GRCh38_HG001_SNP_Indel.GIAB_v4.2.1_and_v3.3.2.vcf.gz"
f_bed = "results/GIAB_hetSNPs.bed.gz"
make_het_snp_bed(f_vcf, f_bed)

In [3]:
f_vcf = "../../public/OneKG/NA12878.1kg.GRCh38.reheader.patmat.vcf.gz"
f_bed = "results/1KG_hetSNPs.bed.gz"
make_het_snp_bed(f_vcf, f_bed)

In [10]:
f_vcf = "../../6_nss-pseudobulk-analysis/results/snvs/concated/nanocaller/PacBio.full.vcf.gz"
f_bed = "results/PacBio_hetSNPs.bed.gz"
make_het_snp_bed(f_vcf, f_bed)

In [11]:
f_vcf = "../../6_nss-pseudobulk-analysis/results/snvs/concated/nanocaller/Ultralong.full.vcf.gz"
f_bed = "results/Ultralong_hetSNPs.bed.gz"
make_het_snp_bed(f_vcf, f_bed)

In [12]:
f_vcf = "../../A1_NanoStrandseqAssembly/results/HG001_Cell_350/round2/snvs.vcf.gz"
f_bed = "results/NSS_hetSNPs_350cells.bed.gz"
make_het_snp_bed(f_vcf, f_bed)